In [1]:
import numpy as np
import json
import csv
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import optuna
%matplotlib inline
np.set_printoptions(precision=3, suppress=True)

In [2]:
def load_data(y):
    path = '../../data/std_data/'
    x_train_std = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).values
    x_test_std = pd.read_pickle(path +'test/{}_x.pkl'.format(str(y))).values
    y_train = pd.read_pickle(path +'train/{}_y.pkl'.format(str(y))).values
    y_test = pd.read_pickle(path +'test/{}_y.pkl'.format(str(y))).values
    features = pd.read_pickle(path +'train/{}_x.pkl'.format(str(y))).columns
    return x_train_std, x_test_std, y_train, y_test, features

In [3]:
def plot_roc_curve(fpr, tpr, auc):
    # ROC曲線をプロット
    plt.plot(fpr, tpr, label='ROC curve (area = %.2f)'%auc)
    plt.legend()
    plt.title('ROC curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.grid(True)
    plt.show()

In [10]:
def lda(trial):
    
    th_prob = 0.5
    
    shrinkage = trial.suggest_uniform('shrinkage', 0, 1)
    
    print("shrinkage: ", shrinkage)
    cm_all = np.zeros((2, 2))
    
    # 予測した確率全体を格納
    probs_all = np.array([])
    y_true_all = np.array([])
    
    for y in range(1978, 2020):
        
        # データの生成
        x_train_std, x_test_std, y_train, y_test, features = load_data(y)
        
        # 学習
        trained = LDA(solver="eigen", shrinkage=shrinkage).fit(x_train_std,  y_train)
        
        # 予測(確率)
        probs = trained.predict_proba(x_test_std)
        probs_all = np.hstack((probs_all, probs[:,1]))
        y_true_all = np.hstack((y_true_all, y_test))
        
        # 混同行列
        y_pred = np.where((probs[:,1] ==  max(probs[:,1])) | (probs[:,1] > th_prob), 1, 0) #確率→0/1
        cm = confusion_matrix(y_true=y_test, y_pred=y_pred)
        cm_all += cm

    auc = roc_auc_score(y_true_all, probs_all)
    fpr, tpr, thresholds = roc_curve(y_true_all, probs_all)
    
#     plot_roc_curve(fpr, tpr, auc)
    
    print("len: {0} , {1}".format(len(y_true_all), len(probs_all) ))
    print("confusion_matrix: ")
    print(cm_all)
    print("AUC: ")
    print(auc)

    return -auc

In [11]:
def main():
    study = optuna.create_study()
    study.optimize(lda, n_trials=100)
    print(study.best_trial)

In [12]:
if __name__ == "__main__":
    main()

shrinkage:  0.4968883256324452
len: 211 , 211
confusion_matrix: 
[[142.  27.]
 [ 21.  21.]]
AUC: 
0.6979430825584673


[I 2019-08-23 01:31:27,884] Finished trial#0 resulted in value: -0.6979430825584673. Current best value is -0.6979430825584673 with parameters: {'shrinkage': 0.4968883256324452}.


shrinkage:  0.22462667462506603
len: 211 , 211
confusion_matrix: 
[[142.  27.]
 [ 21.  21.]]
AUC: 
0.666384897154128


[I 2019-08-23 01:31:28,406] Finished trial#1 resulted in value: -0.666384897154128. Current best value is -0.6979430825584673 with parameters: {'shrinkage': 0.4968883256324452}.


shrinkage:  0.19733108247907616
len: 211 , 211
confusion_matrix: 
[[142.  27.]
 [ 22.  20.]]
AUC: 
0.665257819103973


...

shrinkage:  0.8538602187834515
len: 211 , 211
confusion_matrix: 
[[145.  24.]
 [ 21.  21.]]
AUC: 
0.7324598478444633


[I 2019-08-23 01:32:26,378] Finished trial#98 resulted in value: -0.7324598478444633. Current best value is -0.7424626655395886 with parameters: {'shrinkage': 0.9938100726164629}.


shrinkage:  0.93021352343778
len: 211 , 211
confusion_matrix: 
[[147.  22.]
 [ 21.  21.]]
AUC: 
0.7376725838264299


[I 2019-08-23 01:32:26,881] Finished trial#99 resulted in value: -0.7376725838264299. Current best value is -0.7424626655395886 with parameters: {'shrinkage': 0.9938100726164629}.


FrozenTrial(number=21, state=<TrialState.COMPLETE: 1>, value=-0.7424626655395886, datetime_start=datetime.datetime(2019, 8, 23, 1, 31, 38, 928883), datetime_complete=datetime.datetime(2019, 8, 23, 1, 31, 39, 431084), params={'shrinkage': 0.9938100726164629}, distributions={'shrinkage': UniformDistribution(low=0, high=1)}, user_attrs={}, system_attrs={'_number': 21}, intermediate_values={}, params_in_internal_repr={'shrinkage': 0.9938100726164629}, trial_id=21)


# 結果
shrinkage = 1 or "auto"